This part is just for all the necessary classes


In [10]:
import numpy as np

def softmax(logits):
        exps = np.exp(logits - np.max(logits))
        return exps / np.sum(exps)

def to_one_hot(y, num_classes):
        one_hot = np.zeros(num_classes)
        one_hot[y] = 1
        return one_hot

# Exception if there is a mismatch
class Mismatch(Exception):
    def __init__(self, message):
        super().__init__(message)

# Exception for bias isn't found
class NoBias(Exception):
    def __init__(self, message):
        super().__init__(message)

class Neuron:
    # Neuron Constructor
    def __init__(self):
        # Random until tuned by backpropagation
        self.bias = np.random.randn()
        self.output = 0
        self.input = []


    # Set  bias
    def set_bias(self, bias):
        self.bias = bias
    
    # Get  bias
    def get_bias(self):
        return self.bias


class Layer:
    # Layer Constructor
    def __init__(self, size, input_size):
        self.size = size
        self.input_size = input_size
        self.neurons = [Neuron() for _ in range(size)]
        self.weights = np.random.randn(size, input_size)
        self.input = None  # Placeholder for input data

    # Function used to change the weights of the neurons in a layer
    def change_weights(self, changes):
        if changes.shape != self.weights.shape:
            raise Mismatch("Weight changes shape does not match the weights shape")
        self.weights = changes

    # Function used to change the biases of the neurons in a layer
    def change_biases(self, changes):
        if len(changes) != len(self.neurons):
            raise Mismatch("Bias changes length does not match the number of neurons")
        for i in range(len(self.neurons)):
            self.neurons[i].set_bias(changes[i])

    # Gradient descent algorithm to apply backpropagation
    def gradient_descent(self, errors, lr):
        # Convert input to a NumPy array if it isn't already
        inputs = np.array(self.input)
        
        # Calculate gradients
        dw = np.outer(errors, inputs)  # Shape: (num_neurons, num_inputs)
        db = errors  # Shape: (num_neurons,)
        
        # Gradient Clipping
        dw = np.clip(dw, -1, 1)
        db = np.clip(db, -1, 1)
        
        # Change the weight towards the gradient descent
        self.weights -= lr * dw
        
        # Change the bias towards the gradient descent
        for i, neuron in enumerate(self.neurons):
            neuron.set_bias(neuron.get_bias() - lr * db[i])

    # Give input to a layer
    def give_input(self, input_data):
        if len(input_data) != self.input_size:
            raise Mismatch("Input size does not match the expected input size")
        self.input = input_data
    # Use the formula output= (input*weight)+bias
    # Function to calculate the outputs of the neurons
    def calculate_outputs(self):
        inputs = np.array(self.input)
        weights = np.array(self.weights)
        biases = np.array([neuron.get_bias() for neuron in self.neurons])
        outputs = np.dot(weights, inputs) + biases
        return outputs

    # Function to get the outputs of the layer
    def get_output(self):
        return self.calculate_outputs().tolist()

    


class NN:
    # Neural network constructor
    def __init__(self, input_size, hidden_layers_sizes, output_size):
        self.input_size = input_size
        self.hidden_layers_sizes = hidden_layers_sizes
        self.output_size = output_size
        #Input layer isn't an actual layer btw it's just a placeholder for inputs to go to hidden layer
        #No calculations happen in the input layer
                
        # Initialize hidden layers
        self.hidden_layers = []
        for size in hidden_layers_sizes:
            self.hidden_layers.append(Layer(size, input_size))
            input_size = size
        
        # Initialize output layer
        self.output_layer = Layer(output_size, input_size)

    # Forward Propagation algorithm
    # It's simply giving one layer's output as another layer's input
    def forward_propagation(self, input_data):
        input_O=input_data
        temp = np.array(input_O).flatten()
        for layer in self.hidden_layers:
            layer.give_input(temp)
            temp = layer.get_output()
        hidden_O = temp
        self.output_layer.give_input(hidden_O)
        output_O = self.output_layer.get_output()
        prediction=softmax(output_O)
        return prediction

    # Backward Propagation algorithm
    def back_propagation(self, actual, input_data, lr):
        output = self.forward_propagation(input_data)
        actual=to_one_hot(actual,len(output))
        
        # Derivative of mean squared error formula
        Error_O = 2 * (output - actual) / len(output)

        Hidden_weights = self.output_layer.weights.T
        self.output_layer.gradient_descent(Error_O, lr)

        Error_H_end = np.dot(Hidden_weights, Error_O)
        temp_E = Error_H_end

        for layer in reversed(self.hidden_layers):
            weights = layer.weights.T
            layer.gradient_descent(temp_E, lr)
            temp_E = np.dot(weights, temp_E)

        Input_weights = self.hidden_layers[0].weights
 

    # Training method
    def train(self, training, answers, epoch, lr):
        for e in range(epoch):
            for input_data, expected_output in zip(training, answers):
                self.back_propagation(expected_output, input_data, lr)
            print(f"Epoch {e+1}/{epoch} completed")

    # Testing method
    def test(self, test_data, test_answers):
        correct_predictions = 0
        for input_data, expected_output in zip(test_data, test_answers):
            output = self.forward_propagation(input_data)
            predicted_class = np.argmax(output)  # Get the predicted class
            if predicted_class == expected_output:
                correct_predictions += 1
        accuracy = correct_predictions / len(test_data)
        return accuracy

In [2]:



# Example usage
input_size = 576  # Number of features
hidden_size = [64,32,16,8]  # Number of neurons in hidden layers
output_size = 2  # Number of neurons in output layer

# Create a neural network
nn = NN(input_size=input_size, hidden_layers_sizes=hidden_size, output_size=output_size)

# Example training data
training_data = [

]

# Extend training data with random values
for i in range(10000):

    temp = []
    for i in range(input_size):
        temp.append(np.random.randn())
    training_data.append(temp)

# Example expected outputs
answers = []

# Extend answers with random binary values
for i in range(10000):
    temp = [np.random.choice([0, 1])]
    print(temp)
    answers.append(temp)

# Train the network
epochs = 3
learning_rate = 0.0001
nn.train(training_data, answers, epoch=epochs, lr=learning_rate)
nn.test(training_data, answers)




NameError: name 'NN' is not defined

In [1]:
from keras.datasets import mnist

In [2]:
(test_X, test_y), (train_X, train_y) = mnist.load_data()

In [3]:
train_X = train_X.reshape((train_X.shape[0], 28*28)).astype('float32')
test_X= test_X.reshape((test_X.shape[0], 28*28)).astype('float32')


In [7]:
train_X=train_X/255
test_X=test_X/255
test_X.shape[0]

60000

In [17]:
input_size = 28*28  # Number of features
hidden_size = [128,64,36,18,10]  # Number of neurons in hidden layers
output_size = 10
epochs = 8
learning_rate = 0.001
nn = NN(input_size=input_size, hidden_layers_sizes=hidden_size, output_size=output_size)

training_data=train_X
answers=train_y
testing_data=test_X
test_answers=test_y
print("Test before training")

accuracy=nn.test(testing_data, test_answers)
print(accuracy)

print("=============Training===========")
nn.train(training_data, answers, epoch=epochs, lr=learning_rate)

print("Test after training")
nn.test(testing_data, test_answers)

Test before training
0.12646666666666667
=============Training===========
Epoch 1/8 completed
Epoch 2/8 completed
Epoch 3/8 completed
Epoch 4/8 completed
Epoch 5/8 completed
Epoch 6/8 completed
Epoch 7/8 completed
Epoch 8/8 completed
Test after training


0.8362